# Features

## Question:We want to highlight 10 wines to increase our sales. Which ones should we choose and why? 

In [2]:
import sqlite3
connexion = sqlite3.connect("../db/vivino.db")
cursor = connexion.cursor()

In [25]:
try:
    
    cursor.execute("""
    CREATE VIEW top10_wines AS
    SELECT w.name, ROUND((w.ratings_average + v.ratings_average), 3) AS total_score
    FROM wines AS w
    JOIN vintages AS v ON w.id = v.wine_id
    JOIN wineries AS wi ON w.winery_id = wi.id
    ORDER BY v.ratings_average DESC
    LIMIT 10
""")
    connexion.commit()

    print("view created")
except Exception as e:
    print("Error:", e)
    connexion.rollback()  # Rollback changes if an error occurs

view created


In [26]:

cursor.execute("""
    SELECT *
    from top10_wines                   

""")
cursor.fetchall()

[("L'Extravagant de Doisy-Daëne Sauternes", 9.5),
 ('Brunello di Montalcino Riserva', 9.5),
 ('Pauillac (Premier Grand Cru Classé)', 9.4),
 ('Clos du Mesnil', 9.5),
 ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 9.5),
 ('Amarone della Valpolicella Monte Lodoletta', 9.5),
 ('Harlan Estate Red', 9.5),
 ('Clos du Mesnil', 9.5),
 ('Vintage Port Nacional', 9.4),
 ('Pomerol', 9.5)]

## Question2: We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

In [43]:
try:
    cursor.execute("""
    CREATE VIEW top_keywords AS
    SELECT w.name AS wine_name,
        GROUP_CONCAT(DISTINCT k.name) AS keywords
    FROM wines AS w
    JOIN keywords_wine AS kw ON w.id = kw.wine_id
    JOIN keywords AS k ON kw.keyword_id = k.id
    WHERE k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
    GROUP BY w.name
    HAVING MIN(kw.count) >= 10;
    """)
    connexion.commit()

    print("view created")
except Exception as e:
    print("Error:", e)
    connexion.rollback()  # Rollback changes if an error occurs

view created


In [44]:

cursor.execute("""
    SELECT *
    from top_keywords                   

""")
cursor.fetchall()

[('Almaviva', 'coffee,toast,cream'),
 ("Avizoise Blanc de Blancs Extra Brut Champagne Grand Cru 'Avize'",
  'cream,toast,green apple,citrus'),
 ('Belle Epoque Brut Champagne', 'green apple,toast,coffee,citrus,cream'),
 ('Belondrade y Lurton', 'green apple,cream,toast,citrus'),
 ('Blanc des Millénaires', 'toast,coffee,cream,citrus,green apple'),
 ("Bricco dell'Uccellone Barbera d'Asti", 'coffee,toast,cream'),
 ('Brut Champagne', 'toast,coffee,cream,green apple,citrus'),
 ('CASK 23 Cabernet Sauvignon', 'coffee,toast,cream'),
 ('Campolongo di Torbe Amarone della Valpolicella Classico', 'coffee'),
 ('Chassagne-Montrachet', 'cream,toast,citrus,green apple'),
 ('Comtes de Champagne Blanc de Blancs',
  'toast,coffee,cream,citrus,green apple'),
 ('Corton-Charlemagne Grand Cru Blanc', 'cream,toast,citrus,green apple'),
 ('Cristal Brut Champagne (Millésimé)',
  'toast,coffee,green apple,citrus,cream'),
 ('Cuvée des Enchanteleurs Brut Champagne',
  'toast,coffee,cream,green apple,citrus'),
 ('Dom